In [1]:
!date

Thu Mar  4 23:28:03 IST 2021


In [2]:
import pandas as pd
import numpy as np
import os

import sys
sys.path.insert(0,"..")
from src_dual.EIT_kernel import performance

### Define required functions

In [3]:
def read_reduced_result(output,bucket,file_indent):
    if bucket>=1:
        file=output+f"/EIT_bucket_{int(bucket)}_result_index_{file_indent}.csv"
    else:
        if output.split("/")[-2] == "dual":
            file=output+f"/EIT_dual_kernel_result_index_{file_indent}.csv"
        else:
            file=output+f"/EIT_kernel_result_index_{file_indent}.csv"
    result=pd.read_csv(file)
    return (result)

In [4]:
def transpose_to_unreduced(df,C_unreduced,T_unreduced):
    output=df["output"]
    bucket=df["bucket"]
    file_indent=df["file"]
    z_value=df["z_value"]
    T=df["T"]
    if np.isnan(z_value):
        return (None,None,None,None,None)
    else:
        #Step 1: Read required files
        reduced_result=read_reduced_result(output,bucket,file_indent)
        price_reduced=pd.read_csv(f"../input/index-weekly-data/index_{file_indent}.csv")
        price_not_reduced=pd.read_csv("../input/index-weekly-data/index_1.csv")
        #Step_2: Get Weights
        q_T = price_reduced[reduced_result.security].iloc[T]
        w = reduced_result["X_1"].values * q_T.values
        w = (w / np.sum(w))
        #Step 3: Get new X_1
        q_T_new = price_not_reduced[reduced_result.security].iloc[200]
        X_1_new = w *q_T_new
        new_result = reduced_result.copy()
        new_result.index=new_result.security
        new_result["X_1"] = (w * C_unreduced) / q_T_new
        #Generate performance
        in_excess_return, in_tr_err, out_excess_return, out_tr_err, portfolio_size = \
            performance(price_not_reduced, new_result, C=C_unreduced, T=T_unreduced)
        return(in_excess_return, in_tr_err, out_excess_return, out_tr_err, portfolio_size)

In [5]:
def transpose_df(super_df,C_unreduced,T_unreduced):
    super_df["chk"]=super_df.apply(lambda x: transpose_to_unreduced(x,C_unreduced,T_unreduced),axis=1)
    transpose_cols=["in_excess_return_transpose","in_tr_err_transpose","out_excess_return_transpose",\
               "out_tr_err_transpose","portfolio_size_transpose"]
    super_df[transpose_cols]=pd.DataFrame(super_df["chk"].to_list())
    return (super_df)

### Update the required basic/dual result files

In [6]:
base_path="../eit_basic_dual_unreduced_experiments/"

In [7]:
!ls $base_path

exp_1      exp_1_npca exp_2      exp_2b     exp_2e
exp_1_nmf  exp_1basic exp_2a     exp_2d     exp_3


In [8]:
required_dirs=["exp_1","exp_1_npca","exp_1_nmf"]
for req_dir in required_dirs:
    for key0 in ["dual","basic"]:
        org_df=pd.read_csv(base_path+req_dir+f"/{key0}_results_exp_1.csv")
        key1=req_dir.split("_")[-1]
        key=(key0,key1)
        #Update result
        new_df=transpose_df(org_df,C_unreduced=100000,T_unreduced=200)
        break
        new_df.to_csv(base_path+req_dir+f"/{key0}_results_exp_1_transposed.csv")